In [8]:
import sys
from bb2d import BinaryBoundary2D
from bb2dScript import GetBoundary

sys.version

'3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]'

## シングル版

In [3]:
import socket

s=socket.socket()
port = 13000

s.bind(("localhost",port))
s.listen(5)
c, addr = s.accept()
print("Opponent Address:",addr)
wholeData = b''
for i in range(100):
    recieved = c.recv(3)
    if len(recieved)==0: break
    wholeData += recieved

c.sendall(b'Hello, C#!')

c.close()

print(wholeData)

Opponent Address: ('127.0.0.1', 49832)
b'173,196,1\n133,53,0\n5,243,0\n232,225,1\n160,294,1\n21,174,1\n245,217,1\n156,0,1\n213,267,0\n271,291,1\n162,80,1\n38,299,0\n218,113,1\n169,269,0\n123,180,0\n22,233,1\n160,168,1\n35,40,1\n196,16,1\n5,219,0\n267,34,1\n183,69,0\n105,121,1\n146,102,0\n44,76,1\n58,32,0\n103,191,0\n202,110,0\n9,126,0\n180,142,1\n235,212,1\n168,201,1\n15'


## TCPServer利用

In [4]:
import socketserver

class Handler(socketserver.BaseRequestHandler):
    def handle(self):
        self.data = b''
        for i in range(100):
            recieved = self.request.recv(3)
            print("recieved:",recieved)
            if len(recieved)==0:break
            self.data += recieved        
        #some process(machine learning)
        print(self.data)
        self.request.sendall(b'Hello, C#!')
        print("data sent.")
        
HOST, PORT = "localhost", 13000
# Create the server, binding to localhost on port 9999
with socketserver.TCPServer((HOST, PORT), Handler) as server:
    server.timeout = 300
    server.allow_reuse_address = True
    server.handle_request()
    #server.serve_forever()

OSError: [WinError 10048] 通常、各ソケット アドレスに対してプロトコル、ネットワーク アドレス、またはポートのどれか 1 つのみを使用できます。

## Threadあり

In [9]:
import socket
import threading
import socketserver
import time

class ThreadedTCPRequestHandler(socketserver.BaseRequestHandler):

    def handle(self):
        currentThread = threading.current_thread()
        print("thread:",currentThread.name)
        self.data = b''        
        for i in range(1000):
            recieved = self.request.recv(256)
            #print("recieved:",recieved)
            if len(recieved)==0:break
            self.data += recieved        
        txt = self.data.decode('utf-8')
        result = GetBoundary(txt)
        print(result)
        self.request.sendall(b'Hello, C#!')
        print("data sent.")
    
class ThreadedTCPServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass

HOST, PORT = "localhost", 13000

server = ThreadedTCPServer((HOST, PORT), ThreadedTCPRequestHandler)

with server:
    ip, port = server.server_address
    
    server_thread = threading.Thread(target=server.serve_forever)
    
    server_thread.daemon = True
    
    try:    
        server_thread.start()

        print("Server by thread start.")
        print(ip,":", port)

        time.sleep(30)
    except KeyboardInterrupt as k:
        print("Keyboard Interrupted")
    except Exception as e:
        print(e.message)
    finally:
        server.shutdown()
        print("Server Shutdown.")
    

Server by thread start.
127.0.0.1 : 13000
thread: Thread-19


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 50624)
Traceback (most recent call last):
  File "C:\Users\a-okane.JSC-LINK\AppData\Local\Continuum\Anaconda3\envs\for_chainer\lib\socketserver.py", line 639, in process_request_thread
    self.finish_request(request, client_address)
  File "C:\Users\a-okane.JSC-LINK\AppData\Local\Continuum\Anaconda3\envs\for_chainer\lib\socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\a-okane.JSC-LINK\AppData\Local\Continuum\Anaconda3\envs\for_chainer\lib\socketserver.py", line 696, in __init__
    self.handle()
  File "<ipython-input-9-cc818297bbc1>", line 18, in handle
    result = GetBoundary(txt)
  File "C:\dev\Anaconda\AjaxAndsocket\bb2dScript.py", line 55, in GetBoundary
    mesh, pred = bb2d.getBoundary(0.015)
NameError: name 'BinaryBoundary2D' is not defined
----------------------------------------


Server Shutdown.


In [ ]:

server.shutdown()

In [ ]:
""""
import socketserver

class MyTCPHandler(socketserver.BaseRequestHandler):
    """
    The request handler class for our server.

    It is instantiated once per connection to the server, and must
    override the handle() method to implement communication to the
    client.
    """

    def handle(self):
        # self.request is the TCP socket connected to the client
        self.data = self.request.recv(1024).strip()
        print("{} wrote:".format(self.client_address[0]))
        print(self.data)
        # just send back the same data, but upper-cased
        self.request.sendall(self.data.upper())

if __name__ == "__main__":
    HOST, PORT = "localhost", 9999

    # Create the server, binding to localhost on port 9999
    with socketserver.TCPServer((HOST, PORT), MyTCPHandler) as server:
        # Activate the server; this will keep running until you
        # interrupt the program with Ctrl-C
        server.serve_forever()
        """"